In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from urllib.request import Request, urlopen
import os
import json
import re
import subprocess

SRC_PATH = '../node'
DATA_PATH = '../data'
RAW_DATA_PATH = os.path.join(DATA_PATH, 'bronze')
SHOWDOWN_TS_URL = 'https://raw.githubusercontent.com/smogon/pokemon-showdown/master/data/'
SHOWDOWN_TEXT_URL = 'https://raw.githubusercontent.com/smogon/pokemon-showdown/refs/heads/master/data/text/'

os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(RAW_DATA_PATH, exist_ok=True)

files = [('pokedex.ts', SHOWDOWN_TS_URL), ('moves.ts', SHOWDOWN_TS_URL), ('moves.ts', SHOWDOWN_TEXT_URL),
         ('typechart.ts', SHOWDOWN_TS_URL), ('learnsets.ts', SHOWDOWN_TS_URL), ('abilities.ts', SHOWDOWN_TEXT_URL),
         ('abilities.ts', SHOWDOWN_TS_URL),
         ('items.ts', SHOWDOWN_TEXT_URL), ('items.ts', SHOWDOWN_TS_URL), ('pokedex.ts', SHOWDOWN_TEXT_URL)]

for (filename, URL) in files:
    filename_output =  filename.replace('.ts', '_text.ts') if 'text' in URL else filename.replace('.ts', '_data.ts')
    with urlopen(Request(URL+filename, headers={'User-Agent': 'Mozilla/5.0'})) as response, open(os.path.join(RAW_DATA_PATH, filename_output), 'wb') as f:
        f.write(response.read())
    

In [2]:
files = [
        {'file': 'abilities_data',
         'var_name': 'Abilities'},

         {'file': 'items_data',
         'var_name': 'Items'},

         {'file': 'learnsets_data',
         'var_name': 'Learnsets'},

         {'file': 'moves_data',
         'var_name': 'Moves'},

        {'file': 'pokedex_data',
         'var_name': 'Pokedex'},
         
        {'file': 'typechart_data',
         'var_name': 'TypeChart'},
        ]

for filename in files:
    input_path = Path(f'../data/iron/{filename["file"]}.ts')
    output_path = Path(f'../data/silver/{filename["file"]}.json')
    export_var_name = filename["var_name"]

    output_path.parent.mkdir(parents=True, exist_ok=True)
    tsx_script = os.path.join(SRC_PATH, "showdown_ts_json.mjs")

    cmd = ["npx", "tsx", str(tsx_script), export_var_name, str(input_path), str(output_path)]
    print("Running:", " ".join(cmd))
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        raise RuntimeError(result.stderr)

Running: npx tsx ../node/showdown_ts_json.mjs Abilities ../data/iron/abilities_data.ts ../data/silver/abilities_data.json
Wrote /Users/Winston/Workspace/VGC-SIM/data/silver/abilities_data.json

Running: npx tsx ../node/showdown_ts_json.mjs Items ../data/iron/items_data.ts ../data/silver/items_data.json
Wrote /Users/Winston/Workspace/VGC-SIM/data/silver/items_data.json

Running: npx tsx ../node/showdown_ts_json.mjs Learnsets ../data/iron/learnsets_data.ts ../data/silver/learnsets_data.json
Wrote /Users/Winston/Workspace/VGC-SIM/data/silver/learnsets_data.json

Running: npx tsx ../node/showdown_ts_json.mjs Moves ../data/iron/moves_data.ts ../data/silver/moves_data.json
Wrote /Users/Winston/Workspace/VGC-SIM/data/silver/moves_data.json

Running: npx tsx ../node/showdown_ts_json.mjs Pokedex ../data/iron/pokedex_data.ts ../data/silver/pokedex_data.json
Wrote /Users/Winston/Workspace/VGC-SIM/data/silver/pokedex_data.json

Running: npx tsx ../node/showdown_ts_json.mjs TypeChart ../data/iron/t